In [1]:
labVersion = 'cs190_week3_v_1_3'
from pyspark.mllib.regression import LinearRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint
from test_helper import Test
import numpy as np

In [2]:
# load testing library
from test_helper import Test
import os.path
baseDir = os.path.join('data')
inputPath = os.path.join('cs190', 'millionsong.txt')
fileName = os.path.join(baseDir, inputPath)

numPartitions = 2
rawData = sc.textFile(fileName, numPartitions)

In [54]:
samplePoints = rawData.take(5)
print samplePoints

[u'2001.0,0.884123733793,0.610454259079,0.600498416968,0.474669212493,0.247232680947,0.357306088914,0.344136412234,0.339641227335,0.600858840135,0.425704689024,0.60491501652,0.419193351817', u'2001.0,0.854411946129,0.604124786151,0.593634078776,0.495885413963,0.266307830936,0.261472105188,0.506387076327,0.464453565511,0.665798573683,0.542968988766,0.58044428577,0.445219373624', u'2001.0,0.908982970575,0.632063159227,0.557428975183,0.498263761394,0.276396052336,0.312809861625,0.448530069406,0.448674249968,0.649791323916,0.489868662682,0.591908113534,0.4500023818', u'2001.0,0.842525219898,0.561826888508,0.508715259692,0.443531142139,0.296733836002,0.250213568176,0.488540873206,0.360508747659,0.575435243185,0.361005878554,0.678378718617,0.409036786173', u'2001.0,0.909303285534,0.653607720915,0.585580794716,0.473250503005,0.251417011835,0.326976795524,0.40432273022,0.371154511756,0.629401917965,0.482243251755,0.566901413923,0.463373691946']


In [57]:
def parsePoint(line):

    tokens = line.split(',')
    label = tokens[0]
    features = tokens[1:]
    return LabeledPoint(label, features)

parsedData = rawData.map(parsePoint)

print parsedData.take(3)

[LabeledPoint(2001.0, [0.884123733793,0.610454259079,0.600498416968,0.474669212493,0.247232680947,0.357306088914,0.344136412234,0.339641227335,0.600858840135,0.425704689024,0.60491501652,0.419193351817]), LabeledPoint(2001.0, [0.854411946129,0.604124786151,0.593634078776,0.495885413963,0.266307830936,0.261472105188,0.506387076327,0.464453565511,0.665798573683,0.542968988766,0.58044428577,0.445219373624]), LabeledPoint(2001.0, [0.908982970575,0.632063159227,0.557428975183,0.498263761394,0.276396052336,0.312809861625,0.448530069406,0.448674249968,0.649791323916,0.489868662682,0.591908113534,0.4500023818])]


In [58]:
#Devide rawData into Traning, Validation and Test
weights = [.8, .1, .1]
seed = 50
parsedTrainData, parsedValData, parsedTestData = parsedData.randomSplit(weights, seed)

In [64]:
# Fit the model with default values
fitModel = LinearRegressionWithSGD.train(parsedTrainData)
print  fitModel

(weights=[348.391703677,312.158507888,303.514705245,268.768326368,260.265535915,321.082923267,345.636059404,369.96940298,414.587178279,328.611497772,389.972179858,348.42792115], intercept=0.0)


In [65]:
# Prediction 
testPoint = parsedTrainData.take(1)[0]

print testPoint.label

testPrediction = fitModel.predict(testPoint.features)

print samplePrediction

2001.0
1961.07799016


In [16]:
# Define a formula to caluculate RMSE(Root Mean Square Error)
def squaredError(label, prediction):
    ##Calculates the the squared error for a single prediction.
    return (label - prediction)**2

def calcRMSE(labelsAndPreds):
    ##Calculates the root mean squared error for an `RDD` of (label, prediction) tuples.
    return np.sqrt(labelsAndPreds
                   .map(lambda (label, prediction): squaredError(label, prediction))
                   .mean())

In [40]:
#Create new RDD with actual label and predicted label 
labelsAndPreds = parsedValData.map(lambda lp: (lp.label, fitModel.predict(lp.features)))

#Calculation RMSE
rmseValLR1 = calcRMSE(labelsAndPreds)
print rmseValLR1

108.398553837


In [67]:
##Grid search
# Values to use when training the linear regression model
miniBatchFrac = 1.0  # miniBatchFraction
regType = 'l2'  # regType
useIntercept = True  # intercept

# Seed of minmum RMSE
min_rmseVal = 10**10

modelRMSEs = []

for grid_alpha in [1e-5,1.0,10,100]:
    for grid_numIters in [50,100,500]:
        for grid_reg in [1e-5,0.0, 1.0]:
            model = LinearRegressionWithSGD.train(parsedTrainData,
                                                  iterations=numIters, 
                                                  step=grid_alpha, 
                                                  miniBatchFraction=miniBatchFrac, 
                                                  initialWeights=None, 
                                                  regParam=reg, 
                                                  regType=regType, 
                                                  intercept=useIntercept)
            
            labelsAndPreds = parsedValData.map(lambda lp: (lp.label, model.predict(lp.features)))
            rmseVal = calcRMSE(labelsAndPreds)
            print "best_alpha:{},best_numIter:{}, best_reg:{}, best_rmseVal:{}".format(grid_alpha, grid_numIters, grid_reg, rmseVal)

best_alpha:1e-05,best_numIter:50, best_reg:1e-05, best_rmseVal:1971.21704067
best_alpha:1e-05,best_numIter:50, best_reg:0.0, best_rmseVal:1971.21704067
best_alpha:1e-05,best_numIter:50, best_reg:1.0, best_rmseVal:1971.21704067
best_alpha:1e-05,best_numIter:100, best_reg:1e-05, best_rmseVal:1971.21704067
best_alpha:1e-05,best_numIter:100, best_reg:0.0, best_rmseVal:1971.21704067
best_alpha:1e-05,best_numIter:100, best_reg:1.0, best_rmseVal:1971.21704067
best_alpha:1e-05,best_numIter:500, best_reg:1e-05, best_rmseVal:1971.21704067
best_alpha:1e-05,best_numIter:500, best_reg:0.0, best_rmseVal:1971.21704067
best_alpha:1e-05,best_numIter:500, best_reg:1.0, best_rmseVal:1971.21704067
best_alpha:1.0,best_numIter:50, best_reg:1e-05, best_rmseVal:108.398553837
best_alpha:1.0,best_numIter:50, best_reg:0.0, best_rmseVal:108.398553837
best_alpha:1.0,best_numIter:50, best_reg:1.0, best_rmseVal:108.398553837
best_alpha:1.0,best_numIter:100, best_reg:1e-05, best_rmseVal:108.398553837
best_alpha:1.0,b

In [1]:
#Fitting with figures fixed by grid search 
Final_model = LinearRegressionWithSGD.train(parsedTrainData,
                                           iterations=10, 
                                           step=500, 
                                           miniBatchFraction=miniBatchFrac, 
                                           initialWeights=None, 
                                           regParam = 1e-05, 
                                           regType=regType, 
                                           intercept=useIntercept)

NameError: name 'LinearRegressionWithSGD' is not defined

In [70]:
#Labels comparison between Final_model and actual data. 
Final_testPoint = parsedTrainData.take(1)[0]
print Final_testPoint.label
print Final_testPoint.features

Final_Prediction = Final_model.predict(Final_testPoint.features)
print Final_Prediction

2001.0
[0.884123733793,0.610454259079,0.600498416968,0.474669212493,0.247232680947,0.357306088914,0.344136412234,0.339641227335,0.600858840135,0.425704689024,0.60491501652,0.419193351817]
-8.51546268703e+32
